In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 24.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 20.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system 

In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer, Trainer, TrainingArguments
from datasets import load_dataset
from torch.utils.data import Dataset
import torch

# 1. Класс для загрузки и обработки OPUS100
class OPUS100Dataset(Dataset):
    def __init__(self, tokenizer, lang_pair="en-ru", split="train", max_length=128, num_samples=100):
        self.tokenizer = tokenizer
        self.max_length = max_length

        # Загрузка датасета OPUS100
        dataset = load_dataset("opus100", lang_pair, split=split)

        # Ограничение количества примеров
        if num_samples is not None:
            dataset = dataset.select(range(min(num_samples, len(dataset))))

        # Подготовка данных в формате диалога
        self.examples = []
        for example in dataset:
            self.examples.append({
                'en': example['translation']['en'],
                'ru': example['translation']['ru']
            })

        print(f"Загружено {len(self.examples)} примеров перевода {lang_pair}")

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, idx):
        example = self.examples[idx]
        text = f"Переведи на русский: {example['en']} ||| {example['ru']}"

        encoding = self.tokenizer(
            text,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': encoding['input_ids'].flatten()
        }

# 2. Загрузка предобученной модели
def load_pretrained_model(model_name="gpt2"):
    print(f"Загрузка модели {model_name}...")
    try:
        tokenizer = GPT2Tokenizer.from_pretrained(model_name)
        tokenizer.pad_token = tokenizer.eos_token

        model = GPT2LMHeadModel.from_pretrained(model_name)
        print(f"Модель {model_name} успешно загружена!")
        return model, tokenizer
    except Exception as e:
        print(f"Ошибка загрузки модели: {e}")
        return None, None

# 3. Тонкая настройка на OPUS100
def fine_tune(model, tokenizer, output_dir="./opus100_finetuned"):
    # Подготовка данных
    train_dataset = OPUS100Dataset(tokenizer, num_samples=100)

    # Параметры обучения
    training_args = TrainingArguments(
        output_dir=output_dir,
        num_train_epochs=3,
        per_device_train_batch_size=8,
        save_steps=500,
        save_total_limit=2,
        learning_rate=5e-5,
        logging_dir='./logs',
        logging_steps=100,
        fp16=torch.cuda.is_available(),
        eval_steps=500,
        warmup_steps=100,
        weight_decay=0.01
    )

    # Обучение
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
    )

    print("Начало тонкой настройки на OPUS100...")
    trainer.train()

    # Сохранение модели
    model.save_pretrained(output_dir)
    tokenizer.save_pretrained(output_dir)
    print(f"Модель сохранена в {output_dir}")

    return model, tokenizer

# 4. Генерация ответов
def generate_translation(model, tokenizer, prompt, max_length=100):
    input_text = f"Переведи на русский: {prompt} |||"

    inputs = tokenizer(
        input_text,
        return_tensors='pt',
        max_length=128,
        truncation=True
    ).to(model.device)

    outputs = model.generate(
        inputs.input_ids,
        attention_mask=inputs.attention_mask,
        max_length=max_length,
        temperature=0.7,
        top_k=50,
        top_p=0.9,
        repetition_penalty=1.2,
        num_return_sequences=1,
        pad_token_id=tokenizer.eos_token_id,
        do_sample=True
    )

    # Извлекаем только перевод (часть после |||)
    full_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    translation = full_text.split("|||")[-1].strip()

    return translation

# 5. Интерактивный режим перевода
def interactive_translation(model, tokenizer):
    print("\nРежим перевода с английского на русский. Введите 'выход' для завершения.")
    while True:
        text = input("Введите английский текст: ")
        if text.lower() in ['выход', 'exit', 'quit']:
            break

        translation = generate_translation(model, tokenizer, text)
        print(f"Перевод: {translation}\n")

# Основная функция
def main():
    # Загрузка предобученной модели
    model, tokenizer = load_pretrained_model("gpt2-medium")  # Можно использовать "gpt2", "gpt2-large"

    if model is None:
        return

    # Тонкая настройка (опционально)
    if input("Выполнить тонкую настройку на OPUS100? (y/n): ").lower() == 'y':
        model, tokenizer = fine_tune(model, tokenizer)

    # Запуск интерактивного перевода
    interactive_translation(model, tokenizer)

if __name__ == "__main__":
    main()

Загрузка модели gpt2-medium...
Модель gpt2-medium успешно загружена!
Выполнить тонкую настройку на OPUS100? (y/n): n

Режим перевода с английского на русский. Введите 'выход' для завершения.
Введите английский текст: hello
Перевод: |- - -||
 Tutorials in Russian, English and German are available at www.youtube . To get your free trial visit http://www of google (http:// Google ) or search for "YouTube" , select the video you wish to watch from here then click on play button as shown below...

Введите английский текст: hi
Перевод: |номотым||
1. My mom and I went to the museum with my dad on our way back from school one day, we saw this amazing painting of a guy in his bathtub that had been painted by an artist named Sergei Prokofiev (1909-1980), who was also known as "The Russian Master Painter". The

Введите английский текст: переведи этот текст. hi
Перевод: > "There is nothing wrong with being a man".|| |<"I have been in love for years."|| >

RAW Paste Data

Введите английский текст: 